In [8]:
import os
import sys
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Ensure project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# Import paths
from src.paths import RAW_AUDIO_DIR, PLOTS_DIR, MFCC_INDIVIDUAL_FROM_AUDIO, SPECTRAL_INDIVIDUAL_FROM_AUDIO, SPECTRAL_FEATURES

print("Raw Audio Directory:", RAW_AUDIO_DIR)
print("Plots Directory:", PLOTS_DIR)
print("MFCC Folders:", len(MFCC_INDIVIDUAL_FROM_AUDIO))
print("Spectral Feature Folders:", len(SPECTRAL_INDIVIDUAL_FROM_AUDIO))

Raw Audio Directory: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\data\raw\audio
Plots Directory: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\plots
MFCC Folders: 13
Spectral Feature Folders: 9


In [10]:
# Load audio files
audio_files = [
    f for f in os.listdir(RAW_AUDIO_DIR) 
    if f.lower().endswith(('.wav', '.mp3', '.flac', '.ogg'))
]

audio_files = sorted(audio_files)
print(f"Found {len(audio_files)} audio files:")
for f in audio_files:
    print(f"  - {f}")

Found 10 audio files:
  - C01.mp3
  - C02.mp3
  - C03.mp3
  - C04.mp3
  - C05.mp3
  - C06.mp3
  - C07.mp3
  - C08.mp3
  - C09.mp3
  - C10.mp3


In [11]:
def extract_comprehensive_features(audio_path, sr=22050, n_mfcc=13):
    """Extract all features directly from audio file"""
    try:
        y, sr = librosa.load(audio_path, sr=sr)
    except Exception as e:
        print(f"Error loading {audio_path}: {e}")
        return None
    
    features = {}
    
    # Temporal features
    features['duration'] = librosa.get_duration(y=y, sr=sr)
    features['rms_energy'] = np.mean(librosa.feature.rms(y=y)[0])
    features['zero_crossing_rate'] = np.mean(librosa.feature.zero_crossing_rate(y=y)[0])
    
    # MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    for i in range(n_mfcc):
        features[f'mfcc_{i+1}_mean'] = np.mean(mfcc[i])
        features[f'mfcc_{i+1}_std'] = np.std(mfcc[i])
        features[f'mfcc_{i+1}_max'] = np.max(mfcc[i])
        features[f'mfcc_{i+1}_min'] = np.min(mfcc[i])
    
    # Spectral features
    spec = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)
    psd = np.mean(spec**2, axis=1)
    
    features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    features['spectral_rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    features['spectral_spread'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    features['spectral_flatness'] = np.mean(librosa.feature.spectral_flatness(y=y))
    features['spectral_skewness'] = skew(psd)
    features['spectral_kurtosis'] = kurtosis(psd)
    features['spectral_std'] = np.std(psd)
    features['spectral_slope'] = np.polyfit(freqs, psd, 1)[0]
    features['spectral_decrease'] = np.mean((psd[1:] - psd[0]) / (freqs[1:] + 1e-8))
    
    # Additional features
    features['crest_factor'] = np.max(np.abs(y)) / (np.sqrt(np.mean(y**2)) + 1e-8)
    features['peak_frequency'] = freqs[np.argmax(psd)]
    
    # Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    for i in range(12):
        features[f'chroma_{i}_mean'] = np.mean(chroma[i])
    
    return features

# Extract features from all audio files
all_features = {}
for audio_file in audio_files:
    audio_path = os.path.join(RAW_AUDIO_DIR, audio_file)
    print(f"Processing: {audio_file}...", end=" ")
    feats = extract_comprehensive_features(audio_path)
    if feats is not None:
        all_features[audio_file] = feats
        print("Done")
    else:
        print("Failed")

print(f"\nSuccessfully extracted features from {len(all_features)} files")

Processing: C01.mp3... Done
Processing: C02.mp3... Done
Processing: C03.mp3... Done
Processing: C04.mp3... Done
Processing: C05.mp3... Done
Processing: C06.mp3... Done
Processing: C07.mp3... Done
Processing: C08.mp3... Done
Processing: C09.mp3... Done
Processing: C10.mp3... Done

Successfully extracted features from 10 files
